In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/IPython/extensi

In [2]:
%matplotlib inline

import os
import csv
import json
import pickle
import pandas as pd
import math

import sklearn
from sklearn import model_selection
from sklearn.decomposition import PCA
import numpy as np
import skimage.transform
from skimage.transform import resize
from skimage import io, exposure, img_as_uint, img_as_float
import scipy.misc
from skimage import io
#from pylab import *
#import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from random import randint
random.seed(123)
np.random.seed(123)

#from PIL import Image
import googlenet
import tensorflow as tf
import theano
import theano.tensor as T
import lasagne
from lasagne.utils import floatX

ERROR (theano.sandbox.gpuarray): pygpu was configured but could not be imported
Traceback (most recent call last):
  File "/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/theano/sandbox/gpuarray/__init__.py", line 17, in <module>
    import pygpu
ImportError: No module named pygpu


In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')
df = pd.read_csv(dir + 'cleaned_reports_edited.csv')

In [4]:
exam_ids = []
for folder in os.listdir(dir+'data/Images'):
    exam_ids.append(str(folder))
    
print len(exam_ids)

330


In [ ]:
hs = []
ws = []
for folder in os.listdir(dir+'data/Images'):
    for image in os.listdir(dir+'data/Images/' + folder):
        if image.endswith('.jpg'):
            im = plt.imread(dir+'data/Images/' + folder + '/' + image)
            h,w,c = im.shape
            hs.append(h)
            ws.append(w)

In [ ]:
print min(hs)
print max(hs)
print min(ws)
print max(ws)

In [ ]:
sample_images = []
views = {}
for eid in exam_ids:
    for image in os.listdir(dir + 'data/Images/' + str(eid)):
        if image.endswith('.jpg'):
            item = {}
            imid = os.path.splitext(image)[0] 
            item['exam id'] = eid
            item['impath'] = eid + '/' + image
            if '_' in image and os.path.exists(dir + 'data/Images/' + str(eid) + '/' + 'SSImages_BB' + '/' + 'ss_' + image):             
                leg = imid.split('_')[-1]
                
                if leg != 'B':
                    view = imid.split('_')[-2]

                    item['imid'] = imid #.split('_')[0]
                    item['leg'] = leg
                    item['view'] = view
                    if view not in views.keys():
                        views[view] = 0
                    views[view] = views[view] + 1

                    f_ = open(dir + 'data/Images/' + str(eid) + '/' + 'SSImages_BB' + '/'  + image + '.txt', 'r')
                    item['bbox'] = f_.readlines()[1].rstrip('\n').split(' ')
                else:
                    continue
            
            else:
                continue


            sample_images.append(item)
print views
print len(sample_images)

In [ ]:
train, test = model_selection.train_test_split(sample_images, train_size=0.8, random_state=42)

print len(train), len(test)

In [ ]:
# Resize, pad, downsample, adjust bounding box coordinates

def prep_image(im, bbox, ds=1):
    im = np.array(im)
    h, w, _ = im.shape
    max_value = np.max(im)

    if w >= h:
        # resize
        basewidth = 256
        wpercent = (basewidth / float(w))
        hsize = int((float(h) * float(wpercent)))
        #print 'Before: ', im.shape
        im = resize(im, (hsize, basewidth))
        #print 'After resize: ', im.shape
        bbox = bbox*wpercent
        
        # pad
        p = int(math.floor(abs(hsize-256)/2))
        im = np.pad(im, ((p,256-hsize-p), (0,0), (0,0)),'constant', constant_values=(max_value, max_value))
        
        #print 'After padding: ', im.shape 
        #adjust bbox coordinates
        #bbox[0] = bbox[0] + p
        bbox[1] = bbox[1] - (hsize-256)/2
        #bbox[2] = bbox[2] + p
        bbox[3] = bbox[3] - (hsize-256)/2

            
    elif h > w:
        # resize
        baseheight = 256
        hpercent = (baseheight / float(h))
        wsize = int((float(w) * float(hpercent)))
        #print 'Before: ', im.shape
        im = resize(im, (baseheight, wsize))
        #print 'After resize: ', im.shape
        bbox = bbox*hpercent
        
        # pad
        p = int(math.floor(abs(wsize-256)/2))
        im = np.pad(im, ((0,0), (p,256-wsize-p), (0,0)),'constant', constant_values=(max_value, max_value))

        #print 'After padding: ', im.shape    
        # adjust bbox coordinates
        bbox[0] = bbox[0] - (wsize-256)/2
        #bbox[1] = bbox[1] + p
        bbox[2] = bbox[2] - (wsize-256)/2
        #bbox[3] = bbox[3] + p

    # Downsample
    im = resize(im, (im.shape[0] / ds, im.shape[1] / ds))
    bbox = bbox*1/ds

    bbox = [int(i) for i in bbox]
    
    return im, bbox

In [ ]:
def next_batch(data, batch_size=100, display_images = False):
    batch = random.sample(data, batch_size)
    images = []
    bboxes = []
    for i in batch:
        if display_images:
            plt.figure()
        
        path = dir + 'data/Images/' + str(i['exam id']) + '/' + 'SSImages_BB' + '/'  + 'ss_' + i['imid'] + '.jpg'
        im = plt.imread(path)
        #print np.amax(im), np.amin(im)
        bbox = np.asarray(i['bbox'], dtype=int)
        
        if display_images:
            #print 'Original: ', im.shape, 'Original bbox: ', i['bbox']
            plt.subplot(1,2,1).set_title('Org. bbox: ' + str(bbox))
            plt.subplot(1,2,1).imshow(im)
            rect1 = patches.Rectangle((bbox[0],bbox[1]),abs(bbox[2]-bbox[0]),abs(bbox[3]-bbox[1]),linewidth=1,edgecolor='r',facecolor='none')
            plt.subplot(1,2,1).add_patch(rect1)
        
        im, bbox = prep_image(im, bbox, ds=4)
        images.append(im)
        bboxes.append(bbox)
        
        if display_images:
            #print 'Cropped: ', im.shape, 'Adjusted bbox: ', bbox
            plt.subplot(1,2,2).set_title('Adj. bbox: ' + str(bbox))
            plt.subplot(1,2,2).imshow(im)
            rect2 = patches.Rectangle((bbox[0],bbox[1]),abs(bbox[2]-bbox[0]),abs(bbox[3]-bbox[1]),linewidth=1,edgecolor='r',facecolor='none')
            plt.subplot(1,2,2).add_patch(rect2)
    
    return np.array(images), np.array(bboxes)

In [ ]:
images, bboxes = next_batch(train, batch_size=184)
print images.shape

## Linear regression model

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 256*256*3])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

W = tf.Variable(tf.zeros([256*256*3,4]))
b = tf.Variable(tf.zeros([4]))

y = tf.matmul(x,W) + b

loss = tf.reduce_mean(tf.losses.mean_squared_error(y_, y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [ ]:
BS = 20
print_y = tf.Print(y,[y])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        ims, bbs = next_batch(train, batch_size=BS)
        ims_array = np.array(ims)
        ims = np.reshape(ims_array, [BS, 256*256*3])
        train_step.run(feed_dict={x: ims, y_: bbs})
        if i%10==0:
            print loss.eval(feed_dict={x: ims, y_: bbs})

## CNN regression model

In [16]:
#n_input  = 256*256*3
n_output = 4

def conv_simple(_input):
    # Convolutional Layer #1
    _conv1 = tf.layers.conv2d(
        inputs=_input,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    _pool1 = tf.layers.max_pooling2d(inputs=_conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    _conv2 = tf.layers.conv2d(
        inputs=_pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    _pool2 = tf.layers.max_pooling2d(inputs=_conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    #_pool2_flat = tf.reshape(_pool2, [-1, 5242880])
    
    _pool2_flat = tf.reshape(
        _pool2,
        [-1, _pool2.get_shape().as_list()[1] *
         _pool2.get_shape().as_list()[2] *
         _pool2.get_shape().as_list()[3]])

    #_pool1_flat = tf.reshape(_pool1, [-1, 524288])
    
    _dense = tf.layers.dense(inputs=_pool2_flat, units=1024)#, activation=tf.nn.relu)

    # Output Layer
    _out = tf.layers.dense(inputs=_dense, units=4)
    
    # Return everything
    out = {
        #'input': _input, 'conv1': _conv1, 'conv2': _conv2, 
        #'pool1': _pool1,
        #'pool2': _pool2, 
        #'dense': _dense, 
        'out': _out
    }
    return out

In [17]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 64, 64, 3])
y = tf.placeholder(tf.float32, [None, n_output])

# Parameters
learning_rate   = 0.0001
training_epochs = 100
batch_size      = 20
display_step    = 10

# Functions

_pred = conv_simple(x)['out']
cost = tf.reduce_mean(tf.losses.mean_squared_error(y, _pred))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
#accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
#init = tf.initialize_all_variables()

# Saver 
#save_step = 10;
#savedir = "nets/"
#saver = tf.train.Saver(max_to_keep=3) 

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(train, batch_size)
            # Fit training using batch data
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch

        # Display logs per epoch step
        if epoch % display_step == 0: 
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            #train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
            print (" Training cost: %.3f" % (avg_cost))
            testimgs, testbboxs = next_batch(test, batch_size=47)
            test_cost = sess.run(cost, feed_dict={x: testimgs, y: testbboxs})
            print (" Test cost: %.3f" % (test_cost))

        # Save Net
        if epoch % save_step == 0:
            saver.save(sess, "nets/cnn_bboxes_simple.ckpt-" + str(epoch))

In [15]:
# tf Graph input
#x = tf.placeholder(tf.float32, [None, 64, 64, 3])
#y = tf.placeholder(tf.float32, [None, n_output])

#tf.reset_default_graph()
new_saver = tf.train.Saver() 

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    #new_saver = tf.train.import_meta_graph('nets/cnn_bboxes_simple.ckpt-999.meta')
    new_saver.restore(sess, "nets/64x64/cnn_bboxes_simple.ckpt-999")    
    print("Model restored.")
    #graph = tf.get_default_graph()
    # Check the values of the variables
    testimgs, testbboxs = next_batch(test, batch_size=47)
    test_cost = sess.run(cost, feed_dict={x: testimgs, y: testbboxs})
    print test_cost
    pred_bboxes = sess.run(_pred, feed_dict={x: testimgs, y: testbboxs})
    print pred_bboxes


ValueError: No variables to save

In [ ]:
print len(sample_images)

In [11]:
def resize_pad_image(im):
    im = np.array(im)
    h, w, _ = im.shape
    max_value = np.max(im)

    if w >= h:
        # resize
        basewidth = 256
        wpercent = (basewidth / float(w))
        hsize = int((float(h) * float(wpercent)))
        #print 'Before: ', im.shape
        im = resize(im, (hsize, basewidth))
        #print 'After resize: ', im.shape

        # pad
        p = int(math.floor(abs(hsize-256)/2))
        im_padded = np.pad(im, ((p,256-hsize-p), (0,0), (0,0)),'constant', constant_values=(max_value, max_value))

    elif h > w:
        # resize
        baseheight = 256
        hpercent = (baseheight / float(h))
        wsize = int((float(w) * float(hpercent)))
        #print 'Before: ', im.shape
        im = resize(im, (baseheight, wsize))
        #print 'After resize: ', im.shape

        # pad
        p = int(math.floor(abs(wsize-256)/2))
        im_padded = np.pad(im, ((0,0), (p,256-wsize-p), (0,0)),'constant', constant_values=(max_value, max_value))
    return im_padded

In [12]:
def downsample_reshape_image(im, ds=4):
    im_ds = resize(im, (im.shape[0] / ds, im.shape[1] / ds))
    im_ds = im_ds.reshape((1, im_ds.shape[0], im_ds.shape[1], im_ds.shape[2]))
    return im_ds

In [13]:
def extract_bbox(im, ds=4, display_image = False):
    new_saver = tf.train.Saver() 
    with tf.Session() as sess:
        
        # Restore variables from disk.
        new_saver.restore(sess, "nets/64x64/cnn_bboxes_simple.ckpt-999")    
        #print("Model restored.")

        im_padded = resize_pad_image(im)
        #print im_padded.shape
        im_ds = downsample_reshape_image(im_padded)

        pred_bbox = sess.run(_pred, feed_dict={x: im_ds})
        pred_bbox = [int(i) for i in pred_bbox[0]]
        pred_bbox = np.array(pred_bbox)
        lowerBound, upperBound = 0, 256
        np.clip(pred_bbox, lowerBound, upperBound, out=pred_bbox)

        up_bbox = np.array(pred_bbox)*ds
        np.clip(up_bbox, lowerBound, upperBound, out=up_bbox)
        #print pred_bbox, up_bbox

        if display_image:
            plt.figure()
            #print 'Original: ', im.shape, 'Original bbox: ', i['bbox']
            im_ds = im_ds.reshape((im_ds.shape[1], im_ds.shape[2], im_ds.shape[3]))
            plt.subplot(1,2,1).set_title('Org. bbox: ' + str(pred_bbox))
            plt.subplot(1,2,1).imshow(im_ds)
            rect1 = patches.Rectangle((pred_bbox[0],pred_bbox[1]),
                                      abs(pred_bbox[2]-pred_bbox[0]),
                                      abs(pred_bbox[3]-pred_bbox[1]),linewidth=1,
                                      edgecolor='r',facecolor='none')
            plt.subplot(1,2,1).add_patch(rect1)
            plt.figure()
            plt.subplot(1,2,1).imshow(im_padded)
            rect2 = patches.Rectangle((up_bbox[0],up_bbox[1]),
                                      abs(up_bbox[2]-up_bbox[0]),
                                      abs(up_bbox[3]-up_bbox[1]),linewidth=1,
                                      edgecolor='r',facecolor='none')
            plt.subplot(1,2,1).add_patch(rect2)

    return up_bbox

## Test BBox extractor

In [ ]:
counter=0
for eid in exam_ids:
    print eid
    counter = counter+1
    print counter
    if counter==2:
        break
    for image in os.listdir(dir + 'data/Images/' + str(eid)):
        if image.endswith('.jpg'):
            im = plt.imread(dir + 'data/Images/' + str(eid) + '/' + image)
            bbox = extract_bbox(im, display_image=True)
            print bbox
            im_padded = resize_pad_image(im)
            im_bbox = im_padded[bbox[1]:bbox[3], bbox[0]:bbox[2],:]
            plt.figure()
            plt.imshow(im_bbox)

## Extract Googlenet features for BBoxes and augment training dataset

In [5]:
cnn_layers = googlenet.build_model()
cnn_input_var = cnn_layers['input'].input_var
cnn_feature_layer = cnn_layers['pool5/7x7_s1']
cnn_output_layer = cnn_layers['prob']

get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [6]:
model_param_values = pickle.load(open('models/blvc_googlenet.pkl'))['param values']
lasagne.layers.set_all_param_values(cnn_output_layer, model_param_values)

In [7]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 224, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (224, w*224/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*224/w, 224), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [8]:
all_exams = pickle.load(open('sample_exams_train_test_split.pkl'))
print len(all_exams)

327


Extract googlenet features of bbox

In [ ]:
counter = 0
for exam in all_exams:
    if counter%10==0:
        print counter
    for image in exam['images']:
        path = dir + 'data/Images/' + image['impath']
        #print path
        im = plt.imread(path) 
        # Extract BBox
        bbox = extract_bbox(im)
        im_padded = resize_pad_image(im)
        im_bbox1 = im_padded[bbox[1]:bbox[3], bbox[0]:bbox[2],:]
    #     plt.figure()
    #     plt.imshow(im_bbox1)
    #     plt.title('BBox1')
        # Extract features
        _, cnn_input = prep_image(im_bbox1)
        features = get_cnn_features(cnn_input)
        #print features.shape
        image['bbox cnn features'] = features

In [ ]:
pickle.dump(all_exams, open('sample_exams_bbox_features.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for exam in all_exams:
    print len(exam['images'])

For each image, extract a bbox, apply random (small) translation and mirror flip to augment the dataset. In addition, randomly shuffle sentences

In [9]:
def shuffle_sentences(tokens):
    if len(tokens) > 1:
        if tokens[0]== '.':
            tokens.pop(0)
        report = ' '.join(tokens)
        #print report
        sentences = report.split('.')
        #print sentences
        random.shuffle(sentences)
        #print sentences
        shuffled_sentences = ' . '.join(sentences)
        shuffled_sentences = shuffled_sentences.split(' ')
        s_sentences = [tok for tok in shuffled_sentences if tok!='']
        if len(s_sentences) > 1:
            if s_sentences[0]== '.':
                s_sentences.pop(0)
            if s_sentences[-1]!= '.':
                s_sentences.append('.')
    else:
        s_sentences = tokens
    return s_sentences

In [ ]:
report = [u'.']#, u'moderate', u'left', u'mild', u'right', u'knee', u'tricompartmental', u'degenerative', u'change', u'.', u'deformity', u'tibia', u'previous', u'fracture', u'.']
shuffled_report = shuffle_sentences(report)
print shuffled_report

In [ ]:
augmented_exams_set = pickle.load(open('augmented_exams_bbox_features.pkl'))
print len(augmented_exams_set)

new_set = []
for i, exam in enumerate(augmented_exams_set):
    #print exam['split']
    if exam['split']== 'train':
        new_set.append(exam)

loaded_ids = []
for exam in new_set:
    loaded_ids.append(exam['patient id'])
loaded_ids = set(loaded_ids)
print len(loaded_ids)

for exam in new_set:
    print exam['split']

augmented_exams_set = new_set

In [18]:
augmented_exams_set = []
loaded_ids = []
for exam in all_exams:
    if exam['patient id'] not in loaded_ids:
        print exam['patient id']
        if exam['split'] == 'train':
            new_exam0 = exam
            new_images0 = []

            new_exam1 = exam
            new_images1 = []

            new_exam2 = exam
            new_images2 = []

            new_exam3 = exam
            new_images3 = []

            for image in exam['images']:
                new_image0 = image
                new_image1 = image
                new_image2 = image
                new_image3 = image

                path = dir + 'data/Images/' + image['impath']
                #print path
                im = plt.imread(path)

                #plt.figure()
                #plt.imshow(im)

                # Extract BBox
                bbox0 = extract_bbox(im)
                im_padded = resize_pad_image(im)
                im_bbox0 = im_padded[bbox0[1]:bbox0[3], bbox0[0]:bbox0[2],:]
            #     plt.figure()
            #     plt.imshow(im_bbox1)
            #     plt.title('BBox1')
                # Extract features
                _, cnn_input0 = prep_image(im_bbox0)
                features0 = get_cnn_features(cnn_input0)
                new_image0['bbox cnn features'] = features0
                new_images0.append(new_image0)
                #print features.shape


                # Mirror flip
                im_bbox1_flip = np.flip(im_bbox0, 1)
            #     plt.figure()
            #     plt.imshow(im_bbox1_flip)
            #     plt.title('BBox1 mirror')
                # Extract features
                _, cnn_input = prep_image(im_bbox1_flip)
                features = get_cnn_features(cnn_input)
                #print features.shape
                new_image1['bbox cnn features'] = features
                new_images1.append(new_image1)

                # Apply random transformation
                rand_trans = np.random.randint(-5, 5, 4)
                bbox2 = bbox0 + rand_trans
                lowerBound, upperBound = 0, 256
                np.clip(bbox2, lowerBound, upperBound, out=bbox2)
                im_bbox2 = im_padded[bbox2[1]:bbox2[3], bbox2[0]:bbox2[2],:]
            #     plt.figure()
            #     plt.imshow(im_bbox2)
            #     plt.title('BBox2')
                # Extract features
                _, cnn_input = prep_image(im_bbox2)
                features = get_cnn_features(cnn_input)
                #print features.shape
                new_image2['bbox cnn features'] = features
                new_images2.append(new_image2)

                # Mirror flip
                im_bbox2_flip = np.flip(im_bbox2, 1)
            #     plt.figure()
            #     plt.imshow(im_bbox2_flip)
            #     plt.title('BBox2 mirror')
                # Extract features
                _, cnn_input = prep_image(im_bbox2_flip)
                features = get_cnn_features(cnn_input)
                #print features.shape
                new_image3['bbox cnn features'] = features
                new_images3.append(new_image3)

            # image transformations
            new_exam0['images'] = new_images0
            augmented_exams_set.append(new_exam0)
            new_exam1['images'] = new_images1
            augmented_exams_set.append(new_exam1)
            new_exam2['images'] = new_images2
            augmented_exams_set.append(new_exam2)
            new_exam3['images'] = new_images3
            augmented_exams_set.append(new_exam3)

            # sentence shuffling
            new_exam00 = new_exam0
            new_exam11 = new_exam1
            new_exam22 = new_exam2
            new_exam33 = new_exam3

            new_exam00['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam00)
            new_exam11['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam11)
            new_exam22['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam22)
            new_exam33['tokens'] = shuffle_sentences(exam['tokens'])
            augmented_exams_set.append(new_exam33)

            print len(augmented_exams_set)
        else:
            new_exam = exam
            new_images = []
            
            for image in exam['images']:
                new_image = image
                path = dir + 'data/Images/' + image['impath']
                #print path
                im = plt.imread(path)
                
                bbox = extract_bbox(im)
                im_padded = resize_pad_image(im)
                im_bbox = im_padded[bbox[1]:bbox[3], bbox[0]:bbox[2],:]
                
                _, cnn_input = prep_image(im_bbox)
                features = get_cnn_features(cnn_input)
                #print features.shape
                new_image['bbox cnn features'] = features
                new_images.append(new_image)
                
            new_exam['images'] = new_images  
            augmented_exams_set.append(exam)

RJ109662085
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999


/vol/medic02/users/ag6516/miniconda/envs/TensorFlow/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
8
RJ109696044
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
16
RJ109727291
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensor

200
RJ109724813
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
208
RJ109665070
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
216
RJ109693569
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.c

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
376
RJ109659609
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
384
RJ109677696
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
392
RJ10

...

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
1920
RJ109731135
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
1928
RJ109667939
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
1936
R

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
2080
RJ109734041
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
2088
RJ109691343
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109704616
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckp

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109669837
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109724399
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109661181
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109662147
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109734433
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO

INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
RJ109714118
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999
INFO:tensorflow:Restoring parameters from nets/64x64/cnn_bboxes_simple.ckpt-999


In [21]:
print len(augmented_exams_set)
loaded_ids = []
for exam in augmented_exams_set:
    loaded_ids.append(exam['patient id'])
loaded_ids = set(loaded_ids)
print len(loaded_ids)

2154
327


In [22]:
pickle.dump(augmented_exams_set, open('augmented_exams_bbox_features.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)